In [2]:
import os
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from ast import literal_eval
import random
import time
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('Data/split/train_chunk1.csv')
df = df.iloc[1:,:]
df.to_csv('Data/split/train_chunk1.csv')

C:\Users\samarth\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', 500)

In [8]:
def flatten_df(csv_path='Data/split/train_chunk17.csv'):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'})
    
    for column in JSON_COLUMNS:
        column_as_df = pd.io.json.json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    df['hits'][df['hits']=="[]"] = "[{}]"
    df['hits'] = df['hits'].apply(literal_eval).str[0]
    hits = pd.io.json.json_normalize(df['hits'])
    
    df = pd.concat([df, hits], axis=1, sort=False)    
    return df

##

def clean_df(df):
    df['totals.transactionRevenue'] = df['totals.transactionRevenue'].fillna(0)
    df['totals.totalTransactionRevenue'] = df['totals.totalTransactionRevenue'].fillna(0)
    df['totals.transactions'] = df['totals.transactions'].fillna(0)
    
    keep_cols = ['channelGrouping','fullVisitorId', 'date', 'visitNumber', 'device.browser',
                'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.continent', 'geoNetwork.country', 
                'geoNetwork.region', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 
                'totals.sessionQualityDim', 'totals.timeOnSite', 'totals.totalTransactionRevenue', 
                'totals.transactionRevenue', 'totals.transactions', 'trafficSource.adContent',
                'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.page', 
                'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 
                'trafficSource.source', 'appInfo.exitScreenName', 'contentGroup.contentGroup2', 
                'contentGroup.contentGroup3', 'eventInfo.eventAction']
    df = df[keep_cols]
    
    df = df.replace(['not available in demo dataset', '(not provided)', '(not set)', ''], np.NaN)
    
    return df

##

def feature_date(df):
    year = []
    month = []
    day = []
    
    for each in df['date']:
        each = str(each)
        year.append(int(each[0:4]))
        month.append(int(each[4:6]))
        day.append(int(each[6:8]))
    
    Year = pd.DataFrame(year).rename(columns = {0:'Year'})
    Month = pd.DataFrame(month).rename(columns = {0:'Month'})
    Day = pd.DataFrame(day).rename(columns = {0:'Day'})
    
    df['date'] = df['date'].astype('str')
    df['date'] = pd.to_datetime(df['date'], yearfirst=True)
    
    y = df.index[((df['date'] == '2018-1-1') |
                ((df['date'] > '2018-2-12') & (df['date'] <= '2018-2-19')) |
                ((df['date'] > '2018-5-25') & (df['date'] <= '2018-5-28')) |
                ((df['date'] > '2018-6-29') & (df['date'] <= '2018-7-4')) |
                ((df['date'] > '2018-8-25') & (df['date'] <= '2018-8-31')) |
                ((df['date'] > '2018-9-1') & (df['date'] <= '2018-9-3')) |
                ((df['date'] > '2018-11-20') & (df['date'] <= '2018-11-28')) |
                ((df['date'] > '2018-12-20') & (df['date'] <= '2018-12-31')) |

                (df['date'] == '2017-1-1') |
                ((df['date'] > '2017-2-12') & (df['date'] <= '2017-2-19')) |
                ((df['date'] > '2017-5-26') & (df['date'] <= '2017-5-29')) |
                ((df['date'] > '2017-6-29') & (df['date'] <= '2017-7-4')) |
                ((df['date'] > '2017-8-25') & (df['date'] <= '2017-8-31')) |
                ((df['date'] > '2017-9-2') & (df['date'] <= '2017-9-4')) |
                ((df['date'] > '2017-11-21') & (df['date'] <= '2017-11-29')) |
                ((df['date'] > '2017-12-20') & (df['date'] <= '2017-12-31')) |

                (df['date'] == '2016-1-1') |
                ((df['date'] > '2016-2-12') & (df['date'] <= '2016-2-19')) |
                ((df['date'] > '2016-5-27') & (df['date'] <= '2016-5-30')) |
                ((df['date'] > '2016-6-29') & (df['date'] <= '2016-7-4')) |
                ((df['date'] > '2016-8-25') & (df['date'] <= '2016-8-31')) |
                ((df['date'] > '2016-9-2') & (df['date'] <= '2016-9-5')) |
                ((df['date'] > '2016-11-22') & (df['date'] <= '2016-11-30')) |
                ((df['date'] > '2016-12-20') & (df['date'] <= '2016-12-31')) |

                (df['date'] == '2019-1-1') |
                ((df['date'] > '2019-2-12') & (df['date'] <= '2016-2-19')))]
    
    is_holiday = []
    for index in range(0, df.shape[0]):
        if index in y:
            is_holiday.append(1)
        else:
            is_holiday.append(0)
    
    is_holiday = pd.DataFrame(is_holiday).rename(columns = {0:'is_holiday'})
    df = pd.concat([df, Year, Month, Day, is_holiday], axis = 1)
    df = df.drop(['date'], axis=1)
    
    return df

##

def feature_browser(df):
    browsers = ['Chrome', 'Safari', 'Samsung Internet', 'Firefox', 'Internet Explorer', 'Edge']
    device_browser = []

    for each in df['device.browser']:
        if each in browsers:
            device_browser.append(each)
        elif each == 'Safari (in-app)':
            device_browser.append('Safari')
        elif each == 'Mozilla':
            device_browser.append('Firefox')
        else:
            device_browser.append('Other')

    device_browser = pd.DataFrame(device_browser).rename(columns = {0:'device_browser'})      

    df = pd.concat([df, device_browser], axis = 1)
    df = df.drop(columns=['device.browser'])
    
    return df

##

def feature_inbasket(df):
    basket = []
    for each in df['appInfo.exitScreenName']:
        each = str(each)
        if 'basket' in each:
            basket.append(1)
        else:
            basket.append(0)
    
    is_inbasket = pd.DataFrame(basket).rename(columns = {0:'is_inbasket'})
    
    df = pd.concat([df, is_inbasket], axis = 1)
    df = df.drop(columns=['appInfo.exitScreenName'])
    
    return df

##

def feature_totals(df):
    df['totals.bounces'] = df['totals.bounces'].fillna(0)
    df['totals.newVisits'] = df['totals.newVisits'].fillna(0)
    
    return df

##

def feature_discountad(df):
    discountad = []
    for each in df['trafficSource.adContent']:
        each = str(each)
        if 'discount' in each:
            discountad.append(1)
        else:
            discountad.append(0)
    
    is_discountad = pd.DataFrame(discountad).rename(columns = {0:'is_discountad'})
    
    df = pd.concat([df, is_discountad], axis = 1)
    
    return df

##

def feature_encode(df):
    categorical = ['channelGrouping',
                 'device.deviceCategory',
                 'device.operatingSystem',
                 'geoNetwork.continent',
                 'geoNetwork.country',
                 'geoNetwork.region',
                 'trafficSource.adContent',
                 'trafficSource.adwordsClickInfo.adNetworkType',
                 'trafficSource.adwordsClickInfo.slot',
                 'trafficSource.campaign',
                 'trafficSource.keyword',
                 'trafficSource.source',
                 'contentGroup.contentGroup2',
                 'contentGroup.contentGroup3',
                 'eventInfo.eventAction',
                 'Year',
                 'Month',
                 'Day',
                 'is_holiday',
                 'device_browser',
                 'is_inbasket',
                 'is_discountad']
    numeric = ['visitNumber',
             'totals.bounces',
             'totals.hits',
             'totals.newVisits',
             'totals.pageviews',
             'totals.sessionQualityDim',
             'totals.timeOnSite',
             'totals.transactions',
             'trafficSource.adwordsClickInfo.page']
    
    floatcol = ['totals.totalTransactionRevenue',
             'totals.transactionRevenue']
    
    for each in categorical:
        df[each] = df[each].astype('category')
    
    for each in numeric:
        df[each] = df[each].astype('float32')
    
    for each in floatcol:
        df[each] = df[each].astype('float')
    
    return df

In [9]:
start_time = time.time()
chunk = flatten_df()
flatten_time = time.time()
print('Time to flatten: ', flatten_time - start_time)

chunk = clean_df(chunk)
chunk = feature_date(chunk)
chunk = feature_browser(chunk)
chunk = feature_inbasket(chunk)
chunk = feature_totals(chunk)
chunk = feature_discountad(chunk)
chunk = feature_encode(chunk)
preprocess_time = time.time()
print('Time to process: ', preprocess_time - flatten_time)


train = chunk
# train = pd.concat([train, chunk])
print(train.shape)

C:\Users\samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Time to flatten:  359.2730088233948
Time to process:  10.856256484985352
(99999, 34)


In [10]:
train.head()

,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser,is_inbasket,is_discountad
0,Social,319960211315032594,1.0,desktop,Windows,Asia,Taiwan,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com,NaN,NaN,NaN,2017,4,7,0,Chrome,0,0
1,Social,0877997361530318913,1.0,desktop,Windows,Asia,Turkey,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com,NaN,NaN,NaN,2017,4,7,0,Chrome,0,0
2,Social,5519709534159103194,1.0,desktop,Windows,Asia,Thailand,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com,NaN,NaN,NaN,2017,4,7,0,Chrome,0,0
3,Social,5919139580168518414,1.0,desktop,Windows,Europe,Romania,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com,NaN,NaN,NaN,2017,4,7,0,Chrome,0,0
4,Social,7874806401913518496,1.0,desktop,Linux,Africa,South Africa,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com,NaN,NaN,NaN,2017,4,7,0,Safari,0,0


In [11]:
train.to_csv('Data/train/train17.csv')

In [ ]:
# train_id = train["fullVisitorId"].values